In [12]:
import pandas as pd

folder_dir  = r"/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据"
input_file   = rf"{folder_dir}/extracted_weibo_time.csv"

df = pd.read_csv(input_file, encoding="utf-8",dtype={"省":"string","市":"string","区":"string"},index_col=False )

/var/folders/5h/rb902v091ys_0c5x0jt8sbjw0000gp/T/ipykernel_82720/321748237.py:6: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(input_file, encoding="utf-8",dtype={"省":"string","市":"string","区":"string"},index_col=False )


In [13]:
df

,posts,size_mean,省,市,区,adcode,year,month,day,citycode,time1
0,河南上蔡县为招商引资强征耕地近五百亩。3月20日上午，河南省上蔡县蔡都镇景庄村，绿油油的麦田...,2.520859,河南省,<NA>,上蔡县,411722,2012,3,26,2012-03,['Land/demolition']
1,@柳记七啊九月二十五日凌晨四五点，石狮市锦尚镇港前村的临时灵堂再次被政府强拆，先祖骸骨无处置...,31.584674,福建省,<NA>,石狮市,350581,2014,10,9,2014-10,['Land/demolition']
2,2017年6月5日，江西省分宜县发生群体事件，几百号群众在县政府门口为了新开发的楼盘没有交房...,19.359176,江西省,<NA>,分宜县,360521,2017,6,6,2017-06,['Property']
3,【延安警察暴打嫌犯后夺记者证】3月26日上午，记者采访途经延安市宝塔区桥沟镇政府时，在镇政府...,10.585411,陕西省,<NA>,宝塔区,610602,2014,3,29,2014-03,['Police']
4,#新青年体验营走进特警基地#8月31日，北京通州区青年走进昌平特警基地，观看特警训练、体验惊...,58.729670,北京市,北京市,通州区,110112,2013,9,2,2013-09,[]
...,...,...,...,...,...,...,...,...,...,...,...
34344,爆料 河南省开封市土城四组的村民，近几年来村民来一直生活在惶恐不安的生活中。夷山大厦以及...,NaN,河南省,<NA>,<NA>,410200,2017,4,27,2017-04,['Land/demolition']
34345,2016年年底，吉林市联通开始裁员，某部门裁员比例近半，被裁的百人聚集在市政府门口。突然想起...,45.534698,吉林省,<NA>,<NA>,220200,2017,1,3,2017-01,['Labor']
34346,云南昆明世纪城业主维权被捏造“寻衅滋事”假象，刑拘9名业主的事件，望领导及时对冤案给予昭雪纠...,0.853011,云南省,<NA>,<NA>,530100,2019,6,27,2019-06,['Property']
34347,周五了，要罢工罢工 2南京·南京钟山,2.729843,江苏省,<NA>,<NA>,320100,2017,7,7,2017-07,[]


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import cpca
import os
os.chdir("/Users/anorawu/BFI Dropbox/Wanru Wu/Cloudseeding/data/抗议数据")


df = pd.read_stata("final_panel_newweibo.dta")
df = df.sort_values(by=['adcode', 'date']).reset_index()
df["day"] = df.groupby("adcode").cumcount()
df['event'] = 0

# Identify events (first protest and subsequent protests >= 3 months apart)
for ad, ad_df in df.groupby('adcode'):
    protest_dates = ad_df.loc[ad_df['n_prt_weibo'] > 0, 'day'].sort_values().tolist()
    last_event = None
    
    for protest_date in protest_dates:
        if last_event is None or (protest_date - last_event) >= 45:
            df.loc[(df['adcode'] == int(ad)) & (df['day'] == protest_date),'event'] = 1
            last_event = protest_date

index_list = df.loc[df['event']==1,'index'].tolist()
df['to_day']=None
for index in index_list:
    for i in range(-22,23):
        if not (index+i<0) or (index+i>len(df)):
            df.loc[df['index']==index+i,'to_day'] = i

df.to_csv('eventstudy_weibo_county.csv')